In [1]:
# pip install dash dash-daq plotly shap

In [11]:
df[['SK_ID_CURR']].sample(10)

,SK_ID_CURR
348806,403414
330081,264094
328371,251333
324145,220177
340615,341237
334207,294373
318698,181111
330697,268495
337690,319519
326878,240901


In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import shap
import pandas as pd
# from sklearn.model_selection import train_test_split
import mlflow.sklearn


# Charger le modèle MLFlow
model_path = r"C:\Users\Hankour\OneDrive\Bureau\OC_Arthur\mlruns\159852288404653738\89e2dedfd3dd428b849adecd8c60de14\artifacts\model_lgbm_class_weight_best_model"
model = mlflow.sklearn.load_model(model_path)

# Charger les données 
df = pd.read_pickle('test_df.pkl')

# # Sélection des features
# feats = [f for f in df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
# X = df[feats]

# # Effectuer la prédiction
# predictions = model.predict_proba(X)



#####
# Résultats de la proba via API ? 
# Importation du module , fonction de l'api ?
######
from vs_code_api import predict_proba

id_test = 240901

predict_proba(id_client = id_test)


# Calculer les Shapley values
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)

# Créer le dashboard
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard du Modèle"),
    
    # Sélectionner l'indice de l'observation à afficher
    dcc.Slider(
        id='observation-index',
        min=0,
        max=len(X)-1,
        step=1,
        value=0,
        marks={i: str(i) for i in range(len(X))}
    ),
    
    # Afficher la prédiction du modèle
    html.Div(id='model-prediction'),
    
    # Afficher les Shapley values
    dcc.Graph(id='shap-plot')
])

@app.callback(
    [Output('model-prediction', 'children'),
     Output('shap-plot', 'figure')],
    [Input('observation-index', 'value')]
)
def update_output(selected_index):
    # Prédire la classe pour l'observation sélectionnée
    prediction = model.predict([X.iloc[selected_index]])[0]
    
    # Afficher la prédiction du modèle
    model_prediction_text = f"Prédiction du modèle : {prediction}"
    
    # Visualiser les Shapley values pour l'observation sélectionnée
    shap_plot = px.bar(x=X.columns, y=shap_values[prediction][selected_index])
    shap_plot.update_layout(title=f"Shapley Values pour l'observation {selected_index}")
    
    return model_prediction_text, shap_plot

if __name__ == '__main__':
    app.run_server(debug=True)


c:\Users\Hankour\anaconda3\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning:

Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations



TypeError: string indices must be integers

In [3]:
shap_values

[array([[ 7.18239091e-02, -7.66678436e-03, -8.01477000e-04, ...,
          0.00000000e+00,  0.00000000e+00,  1.65262397e-04],
        [-2.66631698e-01, -2.73583303e-02, -8.01477000e-04, ...,
          0.00000000e+00,  0.00000000e+00,  1.63584583e-04],
        [-1.16675318e-01,  6.04682844e-03, -1.24528906e-02, ...,
          0.00000000e+00,  0.00000000e+00, -1.09414889e-02],
        ...,
        [ 8.40827250e-02,  7.09758116e-03, -7.51999679e-04, ...,
          0.00000000e+00,  0.00000000e+00,  2.67879321e-04],
        [-1.29983826e-01, -1.72792595e-03,  3.80238312e-03, ...,
          0.00000000e+00,  0.00000000e+00,  3.27434967e-04],
        [ 6.05080946e-02,  1.27913573e-02,  2.68929967e-03, ...,
          0.00000000e+00,  0.00000000e+00,  2.67708345e-04]]),
 array([[-7.18239091e-02,  7.66678436e-03,  8.01477000e-04, ...,
          0.00000000e+00,  0.00000000e+00, -1.65262397e-04],
        [ 2.66631698e-01,  2.73583303e-02,  8.01477000e-04, ...,
          0.00000000e+00,  0.00000000e